In [1]:
import requests
import csv
import configparser

In [4]:
# Charger le fichier de configuration
config = configparser.ConfigParser()
config.read('../config/rapid-api.conf')

# Récupérer la valeur de 'myKey' (dans la section DEFAULT)
X_RapidAPI_Key = config['RAPIDAPI']['X-RapidAPI-Key']

In [21]:
import pandas as pd
from io import StringIO


import requests

url = "https://crickbuzz-official-apis.p.rapidapi.com/rankings/batsman/"

querystring = {"formatType":"odi","men":"1"}

headers = {
	"x-rapidapi-key": X_RapidAPI_Key,
	"x-rapidapi-host": "crickbuzz-official-apis.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

# Utiliser StringIO pour lire la réponse comme un fichier CSV
#csv_data = StringIO(response.text)
    
    # Convertir en DataFrame Pandas
df = pd.DataFrame(data = response.json()["rank"])
    
df.head()

,id,rank,name,country,rating,points,lastUpdatedOn,trend,faceImageId,countryId,difference
0,8359,1,Babar Azam,Pakistan,800,800,2024-12-13,Flat,352417,3,NaN
1,576,2,Rohit Sharma,India,765,765,2024-12-13,Flat,577755,2,NaN
2,11808,3,Shubman Gill,India,763,763,2024-12-13,Flat,577753,2,NaN
3,1413,4,Virat Kohli,India,746,746,2024-12-13,Flat,577572,2,NaN
4,11130,5,Harry Tector,Ireland,737,737,2024-12-13,Flat,244641,27,NaN


In [30]:
import requests
import csv
import configparser
import os
from pathlib import Path



CONFIG_FILE= Path(os.getcwd()).resolve().parent / 'config' / 'rapid-api.conf'
DATA_PATH = Path(os.getcwd()).resolve().parent / 'data' 

# Charger le fichier de configuration
config = configparser.ConfigParser()
config.read(CONFIG_FILE)

# Récupérer la valeur de 'X-RapidAPI-Key' (dans la section RAPIDAPI)
X_RapidAPI_Key = config['RAPIDAPI']['X-RapidAPI-Key']
X_RapidAPI_Key

'4efebc1457msh533840ee6a8b9e7p1ed720jsne4646468e9fc'

In [29]:
Path(os.getcwd()).resolve().parent

PosixPath('/Users/belhadri-youcef/Documents/Personnel/Formation/data-projects/crickets-etl-data-project')

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("../data/batsmen_rankings.csv",sep=",")

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15 non-null     object 
 1   rank           15 non-null     object 
 2   name           15 non-null     object 
 3   country        15 non-null     object 
 4   rating         15 non-null     object 
 5   points         15 non-null     object 
 6   lastUpdatedOn  15 non-null     object 
 7   trend          15 non-null     object 
 8   faceImageId    15 non-null     object 
 9   countryId      15 non-null     object 
 10  difference     9 non-null      float64
dtypes: float64(1), object(10)
memory usage: 1.4+ KB


In [7]:
df.shape

(15, 11)

In [20]:
df.query("country == 'England'")


,id,rank,name,country,rating,difference,points,lastUpdatedOn,trend,faceImageId,countryId
2,9419,3,Nat Sciver-Brunt,England,732,-2.0,732,2024-12-13,Down,255048,9
12,9141,13,Tammy Beaumont,England,615,-1.0,615,2024-12-13,Down,227410,9
13,9145,14,Amy Jones,England,596,NaN,596,2024-12-13,Flat,255029,9


## Data Modelisation

In [31]:
# 2. Création des tables de dimensions
## Table `player_dim`
player_dim = df[["id", "name", "faceImageId"]].copy()
player_dim["first_name"] = player_dim["name"].apply(lambda x: x.split(" ")[0])
player_dim["last_name"] = player_dim["name"].apply(lambda x: " ".join(x.split(" ")[1:]))
player_dim.rename(columns={"id": "player_id", "faceImageId": "face_image_id"}, inplace=True)
player_dim = player_dim[["player_id", "first_name", "last_name", "face_image_id"]]

In [13]:
data = df.copy()

In [15]:
player_dim = data[['id', 'name']].copy()  
player_dim['first_name'] = player_dim['name'].apply(lambda x: x.split(" ")[0])  
player_dim['last_name'] = player_dim['name'].apply(lambda x: " ".join(x.split(" ")[1:]))  
player_dim.rename(columns={'id': 'player_id'}, inplace=True)  
player_dim = player_dim[['player_id', 'first_name', 'last_name']]
player_dim.drop_duplicates(inplace=True)# Éviter les doublons  


In [16]:
player_dim.shape

(15, 3)

In [32]:
## Table `country_dim`
country_dim = df[["countryId", "country"]].drop_duplicates().copy()
country_dim.rename(columns={"countryId": "country_id"}, inplace=True)

In [17]:
# Conversion de la colonne 'lastUpdatedOn' en datetime dans le DataFrame source
df['lastUpdatedOn'] = pd.to_datetime(df['lastUpdatedOn'], errors='coerce')

# Création de la table date_dim
date_dim = pd.DataFrame({
    'date': df['lastUpdatedOn'].drop_duplicates()
})
date_dim['date'] = pd.to_datetime(date_dim['date'], errors='coerce')  # Assure la cohérence du type datetime
date_dim['date_id'] = range(1, len(date_dim) + 1)  # Génération d'une clé primaire
date_dim['hour'] = date_dim['date'].dt.hour
date_dim['day_of_week'] = date_dim['date'].dt.day_name()
date_dim['month'] = date_dim['date'].dt.month
date_dim['year'] = date_dim['date'].dt.year


In [ ]:
data['lastUpdatedOn'] = pd.to_datetime(data['lastUpdatedOn'], errors='coerce')  
date_dim = pd.DataFrame({  
            'date_id': range(1, len(data['lastUpdatedOn'].drop_duplicates()) + 1),  
            'date': data['lastUpdatedOn'].drop_duplicates().reset_index(drop=True)  
        })  
date_dim['day_of_week'] = date_dim['date'].dt.day_name()  
date_dim['month'] = date_dim['date'].dt.month  
date_dim['year'] = date_dim['date'].dt.year  

In [18]:
date_dim.head()

,date,date_id,hour,day_of_week,month,year
0,2024-12-27,1,0,Friday,12,2024


In [37]:
# 3. Création de la table de faits (`rankings`)
rankings = df[["id", "countryId", "lastUpdatedOn", "rank", "rating", "points", "difference", "trend"]].copy()
rankings.rename(columns={"id": "player_id", "countryId": "country_id"}, inplace=True)
rankings = rankings.merge(date_dim, left_on="lastUpdatedOn", right_on="date", how="left")
rankings = rankings[["player_id", "country_id", "date_id", "rank", "rating", "points", "difference", "trend"]]


In [42]:
rankings.head()

,player_id,country_id,date_id,rank,rating,points,difference,trend
0,8359,3,1,1,800,800,NaN,Flat
1,576,2,1,2,765,765,NaN,Flat
2,11808,2,1,3,763,763,NaN,Flat
3,1413,2,1,4,746,746,NaN,Flat
4,11130,27,1,5,737,737,NaN,Flat


In [38]:
# 4. Sauvegarde ou visualisation des résultats
print("Player Dimension:\n", player_dim)
print("\nCountry Dimension:\n", country_dim)
print("\nDate Dimension:\n", date_dim)
print("\nRankings Fact Table:\n", rankings)


Player Dimension:
    player_id   first_name       last_name face_image_id
0       8359        Babar            Azam        352417
1        576        Rohit          Sharma        577755
2      11808      Shubman            Gill        577753
3       1413        Virat           Kohli        577572
4      11130        Harry          Tector        244641
5      10713        Daryl        Mitchell        351601
6      13213  Rahmanullah          Gurbaz        352440
7      10384         Shai            Hope        170814
8      13682       Pathum        Nissanka        351889
9      10934      Charith        Asalanka        351873
10      9428      Shreyas            Iyer        352480
11      8497       Travis            Head        332908
12     10209     Heinrich         Klaasen        577487
13      9554       Rassie  van der Dussen        577488
14      8733           KL           Rahul        577534

Country Dimension:
    country_id       country
0           3      Pakistan
1       

In [19]:
data['lastUpdatedOn'] = pd.to_datetime(data['lastUpdatedOn'], errors='coerce')  

date_dim = pd.DataFrame({  
            'date': data['lastUpdatedOn'].drop_duplicates()  
        })  
date_dim['date'] = pd.to_datetime(date_dim['date'], errors='coerce')  
date_dim['date_id'] = range(1, len(date_dim) + 1)  
date_dim['day_of_week'] = date_dim['date'].dt.day_name()  
date_dim['month'] = date_dim['date'].dt.month  
date_dim['year'] = date_dim['date'].dt.year  
date_dim = date_dim[["date_id", "date", "day_of_week", "month", "year"]]

In [21]:
date_dim.head()

,date_id,date,day_of_week,month,year
0,1,2024-12-27,Friday,12,2024


In [23]:
new_dates = date_dim[~date_dim['date'].isin([])].drop_duplicates(subset=['date'])  


In [29]:
date_dim = pd.DataFrame({  
            'date': df['lastUpdatedOn'].drop_duplicates()  
        })  
        # Conversion explicite des dates vers un format compatible avec BigQuery  
        #date_dim['date'] = date_dim['date'].dt.to_pydatetime()  
        # Formatage pour BigQuery (au format chaîne 'YYYY-MM-DD')  
date_dim['date_id'] = range(1, len(date_dim) + 1)  
date_dim['day_of_week'] = date_dim['date'].dt.day_name()  
date_dim['month'] = date_dim['date'].dt.month  
date_dim['year'] = date_dim['date'].dt.year  
date_dim['date'] = date_dim['date'].dt.strftime('%Y-%m-%d')  # Formatage pour BigQuery
date_dim = date_dim[["date_id", "date", "day_of_week", "month", "year"]]
date_dim


,date_id,date,day_of_week,month,year
0,1,2024-12-27,Friday,12,2024


In [25]:
for r in new_dates.to_dict(orient='records'):  
            print( {'type': 'date', 'data': r}  )

{'type': 'date', 'data': {'date_id': 1, 'date': Timestamp('2024-12-27 00:00:00'), 'day_of_week': 'Friday', 'month': 12, 'year': 2024}}
